In [8]:
import os
from dotenv import load_dotenv
import requests

load_dotenv()

True

In [11]:
TENANT_ID = os.getenv('TENANT_ID')
CLIENT_ID = os.getenv('CLIENT_ID')
CLIENT_SECRET = os.getenv('CLIENT_SECRET')

token_url = f"https://login.microsoftonline.com/{TENANT_ID}/oauth2/v2.0/token"

data = {
    'grant_type': 'client_credentials',
    'client_id': CLIENT_ID,
    'client_secret': CLIENT_SECRET,
    'scope': 'https://graph.microsoft.com/.default'
}

response = requests.post(token_url, data=data)
access_token = response.json().get('access_token')

print("Access token:", access_token)

Access token: eyJ0eXAiOiJKV1QiLCJub25jZSI6Im1KcUh3UWdwbXJON2F2R2VFNEVkRERKdTRYNFYxZ1U3SjFuYWJiQjRQZEUiLCJhbGciOiJSUzI1NiIsIng1dCI6IkpZaEFjVFBNWl9MWDZEQmxPV1E3SG4wTmVYRSIsImtpZCI6IkpZaEFjVFBNWl9MWDZEQmxPV1E3SG4wTmVYRSJ9.eyJhdWQiOiJodHRwczovL2dyYXBoLm1pY3Jvc29mdC5jb20iLCJpc3MiOiJodHRwczovL3N0cy53aW5kb3dzLm5ldC84YTYwZTJlMC1jMGRlLTQxODItYWZlMy03NzE3NjdlYTYwMmUvIiwiaWF0IjoxNzU0NTcyMDI1LCJuYmYiOjE3NTQ1NzIwMjUsImV4cCI6MTc1NDU3NTkyNSwiYWlvIjoiazJSZ1lMaTdXUzVPVzIvV25KOG5MY3NUUGpTOUF3QT0iLCJhcHBfZGlzcGxheW5hbWUiOiJuOG4tc2hhcmVwb2ludC1pbnRlZ3JhdGlvbiIsImFwcGlkIjoiNTQ5OTIyNmQtYjAxZS00NDE5LThhZTYtMzAxMTIwMzVmNTIwIiwiYXBwaWRhY3IiOiIxIiwiaWRwIjoiaHR0cHM6Ly9zdHMud2luZG93cy5uZXQvOGE2MGUyZTAtYzBkZS00MTgyLWFmZTMtNzcxNzY3ZWE2MDJlLyIsImlkdHlwIjoiYXBwIiwib2lkIjoiMDc2YmYyZDItOWNhMC00NGRiLWE1YzUtZDhkMWQ3YzNmZjRiIiwicmgiOiIxLkFXNEE0T0pnaXQ3QWdrR3Y0M2NYWi1wZ0xnTUFBQUFBQUFBQXdBQUFBQUFBQUFCdUFBQnVBQS4iLCJyb2xlcyI6WyJTaXRlcy5SZWFkLkFsbCIsIlNpdGVzLlJlYWRXcml0ZS5BbGwiLCJGaWxlcy5SZWFkV3JpdGUuQWxsIiwiRmlsZXMuUmVhZC5Bb

In [12]:
site_name = "BDTeste"
folder_path = "teste_markdown"

# Obter site ID
site_resp = requests.get(
    f"https://graph.microsoft.com/v1.0/sites/taticogestao.sharepoint.com:/sites/{site_name}",
    headers={"Authorization": f"Bearer {access_token}"}
)
site_id = site_resp.json().get("id")
print("Site ID:", site_id)

Site ID: taticogestao.sharepoint.com,83b4cda4-f169-49be-bf80-bb3fc551caba,47486a74-1c37-47cc-8226-d2bbb88a587c


In [14]:
# Obter drive ID (biblioteca de documentos padrão)
drive_resp = requests.get(
    f"https://graph.microsoft.com/v1.0/sites/{site_id}/drive",
    headers={"Authorization": f"Bearer {access_token}"}
)
drive_id = drive_resp.json().get("id")
print("Drive ID:", drive_id)

# Obter lista de arquivos na pasta teste_markdown dentro do drive
files_resp = requests.get(
    f"https://graph.microsoft.com/v1.0/drives/{drive_id}/root:/{folder_path}:/children",
    headers={"Authorization": f"Bearer {access_token}"}
)

if files_resp.status_code == 200:
    files = files_resp.json()
    for file in files.get('value', []):
        print(f"Nome: {file['name']}")
        print(f"Link: {file['webUrl']}")
        print("-" * 40)
else:
    print(f"Erro ao listar arquivos: {files_resp.status_code} - {files_resp.text}")

Drive ID: b!pM20g2nxvkm_gLs_xVHKunRqSEc3HMxHgibSu7iKWHwN2pusafJUTZY69oTv2QRm
Nome: FLN.2024.09 - Alto Padrão (Voss) - ARCELOR (aço e arame).pdf
Link: https://taticogestao.sharepoint.com/sites/BDTeste/Documentos%20Compartilhados/teste_markdown/FLN.2024.09%20-%20Alto%20Padr%C3%A3o%20(Voss)%20-%20ARCELOR%20(a%C3%A7o%20e%20arame).pdf
----------------------------------------
Nome: FLN.2025.04 - Médio-Alto (Viva Balneário) - CONCREBRAS.pdf
Link: https://taticogestao.sharepoint.com/sites/BDTeste/Documentos%20Compartilhados/teste_markdown/FLN.2025.04%20-%20M%C3%A9dio-Alto%20(Viva%20Balne%C3%A1rio)%20-%20CONCREBRAS.pdf
----------------------------------------
Nome: ITP.2024.09 - Luxo (Edify One) - Gerdau (incluso corte e dobra).pdf
Link: https://taticogestao.sharepoint.com/sites/BDTeste/Documentos%20Compartilhados/teste_markdown/ITP.2024.09%20-%20Luxo%20(Edify%20One)%20-%20Gerdau%20(incluso%20corte%20e%20dobra).pdf
----------------------------------------
Nome: PÇR.2025.01 - Médio-Alto Padrão (

In [15]:
# Cria a pasta downloads local (se não existir)
local_folder = "downloads"
os.makedirs(local_folder, exist_ok=True)

for file in files.get('value', []):
    file_name = file['name']
    download_url = file.get('@microsoft.graph.downloadUrl')
    if not download_url:
        print(f"Sem link de download para {file_name}")
        continue

    print(f"Baixando {file_name}...")

    # Requisição GET para baixar o arquivo
    file_resp = requests.get(download_url)
    if file_resp.status_code == 200:
        # Salvar arquivo localmente
        local_path = os.path.join(local_folder, file_name)
        with open(local_path, 'wb') as f:
            f.write(file_resp.content)
        print(f"Salvo em: {local_path}")
    else:
        print(f"Erro ao baixar {file_name}: {file_resp.status_code}")

Baixando FLN.2024.09 - Alto Padrão (Voss) - ARCELOR (aço e arame).pdf...
Salvo em: downloads\FLN.2024.09 - Alto Padrão (Voss) - ARCELOR (aço e arame).pdf
Baixando FLN.2025.04 - Médio-Alto (Viva Balneário) - CONCREBRAS.pdf...
Salvo em: downloads\FLN.2025.04 - Médio-Alto (Viva Balneário) - CONCREBRAS.pdf
Baixando ITP.2024.09 - Luxo (Edify One) - Gerdau (incluso corte e dobra).pdf...
Salvo em: downloads\ITP.2024.09 - Luxo (Edify One) - Gerdau (incluso corte e dobra).pdf
Baixando PÇR.2025.01 - Médio-Alto Padrão (Cill Tower) - CERÂMICA TIJOLAR.pdf...
Salvo em: downloads\PÇR.2025.01 - Médio-Alto Padrão (Cill Tower) - CERÂMICA TIJOLAR.pdf
